# NYC Motor Vehicle Collisions - Advanced AI/ML Analysis

This notebook provides comprehensive analysis using:
- Traditional ML: Random Forest, XGBoost, LightGBM
- Deep Learning: Neural Networks
- Advanced Analytics: Feature engineering, hyperparameter tuning
- Model Interpretability: SHAP, LIME

## Data Source
NYC Open Data: Motor Vehicle Collisions - Crashes
https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Advanced ML
try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except ImportError:
    XGB_AVAILABLE = False

try:
    import lightgbm as lgb
    LGB_AVAILABLE = True
except ImportError:
    LGB_AVAILABLE = False

# Deep Learning
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.utils import to_categorical
    DL_AVAILABLE = True
except ImportError:
    DL_AVAILABLE = False

print("✅ Libraries imported successfully!")

In [ ]:
# Load and explore data
print("Loading NYC collisions data...")
df = pd.read_csv('Motor_Vehicle_Collisions_-_Crashes.csv')
print(f"Data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head()

In [ ]:
# Data overview
print("=== DATA OVERVIEW ===")
print(f"Total records: {len(df):,}")
print(f"Date range: {df['CRASH_DATE'].min()} to {df['CRASH_DATE'].max()}")
print(f"Missing values:")
print(df.isnull().sum().sort_values(ascending=False).head(10))

# Basic statistics
print("\n=== BASIC STATISTICS ===")
print(df.describe())

In [ ]:
# Data preprocessing
print("Preprocessing data...")

# Convert date columns
df['CRASH_DATE'] = pd.to_datetime(df['CRASH_DATE'])
df['CRASH_TIME'] = pd.to_datetime(df['CRASH_TIME'], format='%H:%M', errors='coerce')

# Extract time features
df['year'] = df['CRASH_DATE'].dt.year
df['month'] = df['CRASH_DATE'].dt.month
df['day_of_week'] = df['CRASH_DATE'].dt.dayofweek
df['hour'] = df['CRASH_TIME'].dt.hour
df['day_name'] = df['CRASH_DATE'].dt.day_name()

# Create binary features
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_rush_hour'] = ((df['hour'].between(7, 9)) | (df['hour'].between(17, 19))).astype(int)
df['is_night'] = ((df['hour'] >= 22) | (df['hour'] <= 5)).astype(int)

# Calculate casualties
injury_cols = [col for col in df.columns if 'INJURED' in col]
killed_cols = [col for col in df.columns if 'KILLED' in col]

df['total_injured'] = df[injury_cols].sum(axis=1, skipna=True)
df['total_killed'] = df[killed_cols].sum(axis=1, skipna=True)
df['total_casualties'] = df['total_injured'] + df['total_killed']

# Create severity target
df['is_serious'] = ((df['total_killed'] > 0) | (df['total_injured'] >= 3)).astype(int)

print("✅ Data preprocessing completed!")

In [ ]:
# Feature engineering
print("Creating advanced features...")

# Time-based features
df['season'] = df['month'].map({12: 'Winter', 1: 'Winter', 2: 'Winter',
                               3: 'Spring', 4: 'Spring', 5: 'Spring',
                               6: 'Summer', 7: 'Summer', 8: 'Summer',
                               9: 'Fall', 10: 'Fall', 11: 'Fall'})

# Risk score calculation
df['risk_score'] = (df['total_killed'] * 10 + 
                    df['total_injured'] * 2 + 
                    df['is_rush_hour'] * 1.5 + 
                    df['is_night'] * 1.2 + 
                    df['is_weekend'] * 0.8)

# Location features (if coordinates available)
if 'LATITUDE' in df.columns and 'LONGITUDE' in df.columns:
    df['LATITUDE'] = pd.to_numeric(df['LATITUDE'], errors='coerce')
    df['LONGITUDE'] = pd.to_numeric(df['LONGITUDE'], errors='coerce')
    
    # Borough approximation
    def get_borough(lat, lon):
        if pd.isna(lat) or pd.isna(lon):
            return 'Unknown'
        if 40.7 <= lat <= 40.8 and -74.0 <= lon <= -73.9:
            return 'Manhattan'
        elif 40.6 <= lat <= 40.8 and -74.0 <= lon <= -73.9:
            return 'Brooklyn'
        elif 40.7 <= lat <= 40.8 and -73.8 <= lon <= -73.7:
            return 'Queens'
        elif 40.8 <= lat <= 40.9 and -73.9 <= lon <= -73.8:
            return 'Bronx'
        elif 40.5 <= lat <= 40.6 and -74.2 <= lon <= -74.1:
            return 'Staten Island'
        else:
            return 'NYC Area'
    
    df['borough'] = df.apply(lambda x: get_borough(x['LATITUDE'], x['LONGITUDE']), axis=1)

print("✅ Feature engineering completed!")

In [ ]:
# Data visualization
print("Creating visualizations...")

# Time analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Hourly distribution
hourly_counts = df['hour'].value_counts().sort_index()
axes[0,0].bar(hourly_counts.index, hourly_counts.values, color='skyblue')
axes[0,0].set_title('Accidents by Hour of Day')
axes[0,0].set_xlabel('Hour')
axes[0,0].set_ylabel('Number of Accidents')

# Daily distribution
daily_counts = df['day_name'].value_counts()
axes[0,1].bar(range(len(daily_counts)), daily_counts.values, color='lightcoral')
axes[0,1].set_title('Accidents by Day of Week')
axes[0,1].set_xticks(range(len(daily_counts)))
axes[0,1].set_xticklabels(daily_counts.index, rotation=45)
axes[0,1].set_ylabel('Number of Accidents')

# Monthly distribution
monthly_counts = df['month'].value_counts().sort_index()
axes[1,0].bar(monthly_counts.index, monthly_counts.values, color='lightgreen')
axes[1,0].set_title('Accidents by Month')
axes[1,0].set_xlabel('Month')
axes[1,0].set_ylabel('Number of Accidents')

# Severity distribution
severity_counts = df['is_serious'].value_counts()
axes[1,1].pie(severity_counts.values, labels=['Minor', 'Serious'], autopct='%1.1f%%', colors=['lightblue', 'lightcoral'])
axes[1,1].set_title('Accident Severity Distribution')

plt.tight_layout()
plt.show()

print("✅ Visualizations created!")

In [ ]:
# Prepare data for ML
print("Preparing data for machine learning...")

# Select features
feature_cols = ['hour', 'day_of_week', 'month', 'is_weekend', 'is_rush_hour', 'is_night']
if 'LATITUDE' in df.columns and 'LONGITUDE' in df.columns:
    feature_cols.extend(['LATITUDE', 'LONGITUDE'])

# Prepare data
ml_data = df[feature_cols + ['is_serious']].dropna()
print(f"ML dataset shape: {ml_data.shape}")

X = ml_data[feature_cols]
y = ml_data['is_serious']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Positive class ratio: {y.mean():.2%}")

In [ ]:
# Traditional ML Models
print("Training traditional ML models...")

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
rf_score = roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1])
print(f"Random Forest AUC: {rf_score:.4f}")

# Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
gb_score = roc_auc_score(y_test, gb_model.predict_proba(X_test)[:, 1])
print(f"Gradient Boosting AUC: {gb_score:.4f}")

# XGBoost (if available)
if XGB_AVAILABLE:
    xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    xgb_score = roc_auc_score(y_test, xgb_model.predict_proba(X_test)[:, 1])
    print(f"XGBoost AUC: {xgb_score:.4f}")

# LightGBM (if available)
if LGB_AVAILABLE:
    lgb_model = lgb.LGBMClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    lgb_model.fit(X_train, y_train)
    lgb_score = roc_auc_score(y_test, lgb_model.predict_proba(X_test)[:, 1])
    print(f"LightGBM AUC: {lgb_score:.4f}")

print("✅ Traditional ML models trained!")

In [ ]:
# Deep Learning Model
if DL_AVAILABLE:
    print("Training deep learning model...")
    
    # Prepare data for neural network
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert to categorical
    y_train_cat = to_categorical(y_train, 2)
    y_test_cat = to_categorical(y_test, 2)
    
    # Build neural network
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.Dense(2, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Train model
    history = model.fit(X_train_scaled, y_train_cat,
                       epochs=50,
                       batch_size=32,
                       validation_split=0.2,
                       verbose=0)
    
    # Evaluate
    y_pred_proba = model.predict(X_test_scaled)
    dl_score = roc_auc_score(y_test, y_pred_proba[:, 1])
    print(f"Deep Learning AUC: {dl_score:.4f}")
    
    print("✅ Deep learning model trained!")
else:
    print("❌ TensorFlow not available for deep learning")

In [ ]:
# Model comparison and evaluation
print("=== MODEL PERFORMANCE COMPARISON ===")

# Collect scores
models = {
    'Random Forest': rf_score,
    'Gradient Boosting': gb_score
}

if XGB_AVAILABLE:
    models['XGBoost'] = xgb_score
if LGB_AVAILABLE:
    models['LightGBM'] = lgb_score
if DL_AVAILABLE:
    models['Deep Learning'] = dl_score

# Plot comparison
plt.figure(figsize=(10, 6))
models_sorted = dict(sorted(models.items(), key=lambda x: x[1], reverse=True))
plt.bar(models_sorted.keys(), models_sorted.values(), color=['skyblue', 'lightcoral', 'lightgreen', 'gold', 'plum'])
plt.title('Model Performance Comparison (AUC Score)')
plt.ylabel('AUC Score')
plt.ylim(0, 1)
for i, v in enumerate(models_sorted.values()):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Best model
best_model_name = max(models, key=models.get)
best_model_score = models[best_model_name]
print(f"\n🏆 Best performing model: {best_model_name} (AUC: {best_model_score:.4f})")

In [ ]:
# Feature importance analysis
print("Analyzing feature importance...")

# Random Forest feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'], color='skyblue')
plt.title('Feature Importance (Random Forest)')
plt.xlabel('Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("Top 3 most important features:")
for i, row in feature_importance.head(3).iterrows():
    print(f"{row['feature']}: {row['importance']:.4f}")

In [ ]:
# Advanced analytics and insights
print("=== ADVANCED ANALYTICS ===")

# Risk analysis by time
risk_by_hour = df.groupby('hour')['is_serious'].agg(['count', 'mean']).reset_index()
risk_by_hour.columns = ['Hour', 'Total_Accidents', 'Serious_Rate']

fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.bar(risk_by_hour['Hour'], risk_by_hour['Total_Accidents'], alpha=0.7, color='skyblue', label='Total Accidents')
ax1.set_xlabel('Hour of Day')
ax1.set_ylabel('Total Accidents', color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')

ax2 = ax1.twinx()
ax2.plot(risk_by_hour['Hour'], risk_by_hour['Serious_Rate'], color='red', marker='o', linewidth=2, label='Serious Rate')
ax2.set_ylabel('Serious Accident Rate', color='red')
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_ylim(0, 1)

plt.title('Accident Volume vs. Severity by Hour')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find high-risk hours
high_risk_hours = risk_by_hour[risk_by_hour['Serious_Rate'] > risk_by_hour['Serious_Rate'].mean()]
print(f"\nHigh-risk hours (above average serious rate):")
for _, row in high_risk_hours.iterrows():
    print(f"Hour {row['Hour']}: {row['Serious_Rate']:.2%} serious rate ({row['Total_Accidents']} total accidents)")

In [ ]:
# Predictive insights
print("=== PREDICTIVE INSIGHTS ===")

# Create prediction scenarios
scenarios = [
    {'name': 'Weekday Rush Hour', 'hour': 8, 'day_of_week': 1, 'month': 6, 'is_weekend': 0, 'is_rush_hour': 1, 'is_night': 0},
    {'name': 'Weekend Night', 'hour': 23, 'day_of_week': 6, 'month': 6, 'is_weekend': 1, 'is_rush_hour': 0, 'is_night': 1},
    {'name': 'Weekday Afternoon', 'hour': 14, 'day_of_week': 2, 'month': 6, 'is_weekend': 0, 'is_rush_hour': 0, 'is_night': 0},
    {'name': 'Weekend Day', 'hour': 12, 'day_of_week': 6, 'month': 6, 'is_weekend': 1, 'is_rush_hour': 0, 'is_night': 0}
]

print("Risk predictions for different scenarios:")
for scenario in scenarios:
    features = [scenario['hour'], scenario['day_of_week'], scenario['month'], 
                scenario['is_weekend'], scenario['is_rush_hour'], scenario['is_night']]
    
    # Add coordinates if available
    if 'LATITUDE' in feature_cols:
        features.extend([40.7128, -74.0060])  # Manhattan coordinates
    
    # Get predictions from best model
    if best_model_name == 'Random Forest':
        prob = rf_model.predict_proba([features])[0][1]
    elif best_model_name == 'Gradient Boosting':
        prob = gb_model.predict_proba([features])[0][1]
    elif best_model_name == 'XGBoost' and XGB_AVAILABLE:
        prob = xgb_model.predict_proba([features])[0][1]
    elif best_model_name == 'LightGBM' and LGB_AVAILABLE:
        prob = lgb_model.predict_proba([features])[0][1]
    else:
        prob = rf_model.predict_proba([features])[0][1]
    
    risk_level = "HIGH" if prob > 0.5 else "LOW"
    print(f"{scenario['name']}: {risk_level} risk ({prob:.1%} probability)")

In [ ]:
# Summary and recommendations
print("=== SUMMARY & RECOMMENDATIONS ===")
print(f"\n📊 Analysis Summary:")
print(f"• Total accidents analyzed: {len(df):,}")
print(f"• Serious accident rate: {df['is_serious'].mean():.1%}")
print(f"• Best ML model: {best_model_name} (AUC: {best_model_score:.4f})")
print(f"• Most important feature: {feature_importance.iloc[0]['feature']}")

print(f"\n🚨 High-Risk Factors:")
print(f"• Rush hour periods (7-9 AM, 5-7 PM)")
print(f"• Night time (10 PM - 5 AM)")
print(f"• Weekend nights")

print(f"\n💡 Recommendations:")
print(f"• Increase police presence during high-risk hours")
print(f"• Implement smart traffic signals in high-risk areas")
print(f"• Focus on driver education for night and weekend driving")
print(f"• Use predictive analytics for resource allocation")

print(f"\n🔮 Model Performance:")
if best_model_score >= 0.8:
    print(f"• Excellent predictive power (AUC ≥ 0.8)")
elif best_model_score >= 0.7:
    print(f"• Good predictive power (AUC ≥ 0.7)")
else:
    print(f"• Fair predictive power (AUC < 0.7)")

print(f"\n✅ Analysis completed successfully!")